# Get Degree Values and Spanning Tree Constant Values for a Model

This notebook is a streamlined version of `model_deg_investigation.ipynb`. Instead of analyzing the structure of all models, it focuses on a single model at a time. This allows users to quickly examine degree distributions and the average spanning tree constant, making it easier to fine-tune parameters and evaluate how changes impact key structural properties.

In [2]:
# Import Libraries
import importlib
import models
importlib.reload(models)

import pickle
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from statistics import median
import numpy as np
from numpy.linalg import slogdet

In [10]:
# Define necessary command

def max_degree(G):
    # Initialize max_degree
    max_degree = -1

    # Iterate over all nodes and their degrees
    for degree in G.degree():
        if degree[1] > max_degree:
            max_degree = degree[1]
    
    return max_degree

In [11]:
# Determine number of graphs to explore
# by setting number of vertices and random seeds

df = pd.read_csv("csv_files/model_one_desired_avg_deg.csv")
df = df[["num_vertices", "rand_seed"]].drop_duplicates()

new_vert = [200, 400, 600, 800, 1000, 1400, 1800]
for vert in new_vert:
    for seed in df["rand_seed"].unique()[0:3]:
        new_row = pd.DataFrame([{'num_vertices': vert, 'rand_seed': seed}])
        df = pd.concat([df, new_row], ignore_index=True)

In [12]:
# Get values for characteristics of the graphs

planar = []
connected = []
avg_deg = []
median_deg = []
max_deg = []

# Spanning tree information
num_nodes = []
tree_const = []
log_trees = []

for index, row in df.iterrows():
    n = int(row["num_vertices"])
    rs = int(row["rand_seed"])
    
    # CHOOSE MODEL
    model = "12"
    G = models.model_eight_order_switched(n, rs)

    # Planar?
    planar.append(nx.check_planarity(G)[0])
    
    # Connected?
    connected.append(nx.is_connected(G))

    # Calculate avg degree of graph
    avg_deg.append(2 * G.number_of_edges() / G.number_of_nodes())
    
    # Calculate median degree of graph
    degrees = sorted([degree for _, degree in G.degree()], reverse=False)
    median_deg.append(median(degrees))

    # Calculate max degree
    max_deg.append(max_degree(G))
    
    # calculate graph Laplacian
    # print(f'calculating model {model}:', n, rs)
    Lap = nx.laplacian_matrix(G).toarray()
    T = np.delete(Lap,1,0)
    T = np.delete(T,1,1)
    (sign, logabsdet) = slogdet(T)
    if (sign == 1):
        tree_const.append(logabsdet/n)
        num_nodes.append(n)
        log_trees.append(logabsdet)

model_df = df.assign(planar=planar, connected=connected, 
                            avg_deg=avg_deg, median_deg=median_deg, max_deg=max_deg)

In [14]:
# Calculate the average of planar, connected, median_deg, max_deg
columns_avg = model_df[['planar', 'connected', 'avg_deg', 'median_deg', 'max_deg']].mean()
columns_avg["avg_st_con"] = st_cons=np.mean(tree_const)
columns_avg.round(2)

planar         0.00
connected      1.00
avg_deg        5.45
median_deg     5.39
max_deg       10.50
avg_st_con     1.44
dtype: float64

In [15]:
# For real world data:
# avg deg 5.4
# st con 1.33